# 简介
- 原文[How to Grid Search Hyperparameters for Deep Learning Models in Python With Keras](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)

深度学习算法工程师经常被戏称为调参工程师，的确深度学习大部分时间都在调整参数，为了找到最优的参数可真是煞费苦心。

这篇文章就翻译了 Jason Brownlee 博士的使用网格搜索算法来寻找神经网络的最优解。下面是文章主题部分，英文水平有限，难免有不准确的地方还请谅解。英文好的可以看原文，不是特别难，四级的词汇基本上就可以。

超参优化(Hyperparameter optimization ) 占据了深度学习很大的一部分比例。

这是因为，神经网络众所周知的难以配置，它有太多的参数需要设置。最主要的是，个别模型训练起来特别的慢。

在这片文章中，你将学习如何使用 python 机器学习库 scikit-learn 的网格搜索功能，来调节 Keras 深度学习模型的超参。

通过阅读这片文章你将了解：
- 怎么通过包装 Keras 模型来使用 scikit-learn，并学习怎么使用网格搜索（grid search）
- 怎样网格搜索通用神经网络参数，如学习速率（learning rate），丢弃率（dropout rate），迭代次数(epoches)及其神经元的个数
- 在自己的项目上怎么定义超参调节实验。

下面进入正题吧。

# 总览

在这篇文章中，我将向你展示怎么使用 scikit-learn 的网格搜索功能，以及给你一组案例，你可以copy 到自己的项目中作为开始点。

下面列出了我们将要覆盖的主题：
1. 怎么样在 scikit-learn 中使用 Keras 的模型
2. 怎么使用 scikit-learn 的网格搜索
3. 怎么调节 batch size 和训练的迭代次数（training epochs）
4. 如何调节优化算法(optimization algorithms)
5. 如何调节学习速率（learning rate）和动量（momentum）
6. 如何调节网络的初始化权重（network weight initialization）
7. 如何调节激活函数（activation  functions）
8. 如何调节 dropout 正则化（dropout regularization）
9. 如何调节隐藏着的神经元个数

# 在 scikit-learn 中使用 Kears 的模型

通过 Keras 提供的 `KerasClassifier` 与 `KerasRegressor` 类，将 Keras 模型包装起来就可以在 scikit-learn 中使用了。

为了使用这些包装你必须定义一个函数，它用来创建并返回你的 Keras 序列化模型（sequential model），然后创建 `KerasClassifier` 类时将其传给 `build_fn` 函数。例如：
```python
def create_model():
    ...
    return model
model = KerasClassifier(build_fn=create_model)
```

`KerasClassifier` 的构造器可以接收默认参数，这些参数被传递给调用`model.fit()`，如迭代的次数、批次的大小。例如：
```python
def create_model():
    ...
    return model
model = KerasClassifier(build_fn=create_model, epochs=10)
```

`KerasClassifier` 的构造器也可以接收新的参数，它可以被传递给你自己定义的`create_model()`函数。这些新的参数必须定义在你的`create_model()`函数的签名通过默认参数。例如：
```python
def create_model(dropout_rate=0.0):
	...
	return model
model = KerasClassifier(build_fn=create_model, dropout_rate=0.2)
```

通过 [scikit-learn wrapper in Keras API](https://keras.io/scikit-learn-api/) 文档你可以了解更多.

# 如何使用 scikit-learn 中网格搜索

网格搜索是模型超参优化技术。在 scikit-learn 中该技术通过 `GridSearchCV` 类被提供出来。

构建这个类的时候，你必须提供超参的字典来评估`param_grid`参数。他是模型参数名字与一组用于尝试值的映射。默认情况下，accuracy 是被用于优化的分数，但通过 `GridSearchCV` 构造器的 `score` 参数，可以指定其他的得分评估方法。

默认情况下，网格搜索只是用一个线程，通过将 `GridSearchCV` 构造器的 `n_jobs` 参数设置为 -1，进程会使用你机器上的所有内核。依据不 Keras 的后端，这可能会影响神经网络的训练进程。

`GridSearchCV` 进程会为每一个参数组合构建并训练一个模型。交叉验证(Cross validation)被用于评估每一个独立的模型，同时默认的3折交叉验证（3-fold cross validation）被使用，尽管通过 `GridSearchCV`构造器的 `cv` 参数可以重新指定他的值。下面是一个简单的网格搜索:
```python
param_grid = dict(epochs=[10,20,30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
```

一旦完成，你就可以通过`grid.fit()`返回的结果对象访问网格搜索的输出。 `best_score_`成员提供访问在监测优化程序过程中最佳分数，`best_params_` 描述了在或者最好结果时的参数组合。

通过 [ GridSearchCV class in the scikit-learn API ]() 文档你可以了解更多关于 `GridSearchCV` 。

# 问题描述

现在你已经学会了如何在 scikit-learn 中使用 Keras 模型，以及如何在 scikit-learn 中使用网格搜索，让我们看一组案例。

所有的案例都将在一个叫做 [Pima Indians onset of diabetes classification dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database)(原数据连接失效，这里给出的是 Kaggle 上的数据集)的小偏差机器学习数据集上演示。 这个小的数据集都是数字化的属性，因此处理起来更容易。[pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv) 下载地址。

应为在这篇文章我们都要处理这个案例，我们会合并最佳参数。因为参数之间互相影响，所以在不是最佳的网格搜索方法，但是对于示范却很好。

下面所有的案例都配置了并行处理 `n_jobs=-1`。如果你发现了类似下面的错误：
```
INFO (theano.gof.compilelock): Waiting for existing lock by process '55614' (I am process '55613')
INFO (theano.gof.compilelock): To manually release the lock, delete ...
```

那么，停止进行，改变代码，将 `n_jobs=1`，让网格搜索不进行并行处理。 

**注意：以下代码的运行结果有些与原文的结果并不一致**。

# 调节
## batch size 与 epochs

在第一个简单的案例中，我们考虑调节在拟合网络时使用的 batch size 和 epochs。

在迭代梯度下降（[ iterative gradient descent i](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method)）中，batch size 是权重更新前输入网络的样本数。在训练网络时他也是一个优化，他决定了一次有多少样本读入到内存中。

epochs 数是在训练期间，整个训练数据集输入到网络的次数。有些网络对batch size 特别敏感，如 LSTM 循环神经网络（recurrent neural networks）和卷积神经网络（Convolutional Neural Networks）。

这里我们会评估一组不同的最小 batch size 从 10 到 100 每隔 20步，代码如下：

In [12]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# 构建模型的函数
def create_model():
    # 创建模型
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 编译模型
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型
model = KerasClassifier(build_fn=create_model, verbose=0)
# 定义网格搜索参数
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

最终的结果如下：
```
Best: 0.716146 using {'batch_size': 10, 'epochs': 100}
0.583333 (0.030314) with: {'batch_size': 10, 'epochs': 10}
0.691406 (0.008438) with: {'batch_size': 10, 'epochs': 50}
0.716146 (0.010253) with: {'batch_size': 10, 'epochs': 100}
0.605469 (0.016877) with: {'batch_size': 20, 'epochs': 10}
0.348958 (0.024774) with: {'batch_size': 20, 'epochs': 50}
0.348958 (0.024774) with: {'batch_size': 20, 'epochs': 100}
0.348958 (0.024774) with: {'batch_size': 40, 'epochs': 10}
0.638021 (0.029635) with: {'batch_size': 40, 'epochs': 50}
0.641927 (0.028940) with: {'batch_size': 40, 'epochs': 100}
0.626302 (0.033804) with: {'batch_size': 60, 'epochs': 10}
0.654948 (0.009207) with: {'batch_size': 60, 'epochs': 50}
0.688802 (0.021236) with: {'batch_size': 60, 'epochs': 100}
0.662760 (0.003683) with: {'batch_size': 80, 'epochs': 10}
0.549479 (0.159291) with: {'batch_size': 80, 'epochs': 50}
0.572917 (0.134575) with: {'batch_size': 80, 'epochs': 100}
0.630208 (0.040386) with: {'batch_size': 100, 'epochs': 10}
0.527344 (0.143666) with: {'batch_size': 100, 'epochs': 50}
0.438802 (0.122962) with: {'batch_size': 100, 'epochs': 100}
```

我们看到 batch size 为 10，epochs 为100时获得了最佳的结果。

## 优化算法

keras 提供了一套最好的优化算法。在这个案例中，我们调节用于训练网络的优化算法，每一个都是用默认的参数。这个是一个很奇怪的案例，通常你会选择一个方法，而不是在你的问题中聚焦在调节参数上。[Keras](http://keras.io/optimizers/) 提供了其支持的优化算法，代码如下：

In [ ]:
# 构建模型的函数
def create_model(optimizer='adam'):
    # 创建模型
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 编译模型
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型，使用到了上一步找出的 epochs、batch size 最优参数
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=80, verbose=0)
# 定义网格搜索参数
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

运行的结果如下：
```
Best: 0.680990 using {'optimizer': 'Adamax'}
0.660156 (0.030758) with: {'optimizer': 'SGD'}
0.643229 (0.030978) with: {'optimizer': 'RMSprop'}
0.666667 (0.016367) with: {'optimizer': 'Adagrad'}
0.661458 (0.030647) with: {'optimizer': 'Adadelta'}
0.583333 (0.143075) with: {'optimizer': 'Adam'}
0.680990 (0.029635) with: {'optimizer': 'Adamax'}
0.561198 (0.156554) with: {'optimizer': 'Nadam'}
```
结果建议 Adamax 优化算法获得了最好的分数，精确度达到了 68%。

## Learning Rate与Momentum

通常我们会预选一个优化算法来训练我们的网络并调节他的参数。到目前为止最通用的优化算法是[Stochastic Gradient Descent](http://keras.io/optimizers/#sgd) (SGD)，因为他是最容易理解的。在这个案例中我们会优化 SGD 的学习速率和动量参数。

学习速率控制着在每一批次的最后权重更新多少，动量控制着让前一次的更新对现在的权重更新有多少影响。我们会尝试一组小标准差的学习速率和动量，值的范围是 0.2 到 0.8 ，间隔为 0.2，同时也包括 0.9（因为这是在实践中最常用的值）。

通常，一个好的想法是在优化中包括epochs，像这次，因为在每一批学习的量（learning Rate），每一批更新的次数（batch size）和第迭代次数（epochs）是依赖的。示例代码如下：

In [ ]:
from keras.optimizers import SGD
# 构建模型的函数
def create_model(learn_rate=0.01, momentum=0):
    # 创建模型
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 编译模型
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型，使用到了上一步找出的 epochs、batch size 最优参数
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=80, verbose=0)
# 定义网格搜索参数
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

程序运行的结果如下：
```
Best: 0.673177 using {'learn_rate': 0.001, 'momentum': 0.4}
0.652344 (0.030425) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.567708 (0.006639) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.673177 (0.004872) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.656250 (0.012758) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.574219 (0.135579) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.647135 (0.045143) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.542969 (0.145880) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.540365 (0.143075) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.574219 (0.132736) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.548177 (0.148552) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.572917 (0.134575) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.533854 (0.149269) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.649740 (0.026557) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.4}
0.544271 (0.146518) with: {'learn_rate': 0.1, 'momentum': 0.6}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.8}
0.427083 (0.134575) with: {'learn_rate': 0.1, 'momentum': 0.9}
0.572917 (0.134575) with: {'learn_rate': 0.2, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.2, 'momentum': 0.2}
0.651042 (0.024774) with: {'learn_rate': 0.2, 'momentum': 0.4}
0.348958 (0.024774) with: {'learn_rate': 0.2, 'momentum': 0.6}
0.427083 (0.134575) with: {'learn_rate': 0.2, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.2, 'momentum': 0.9}
0.532552 (0.148552) with: {'learn_rate': 0.3, 'momentum': 0.0}
0.541667 (0.145263) with: {'learn_rate': 0.3, 'momentum': 0.2}
0.651042 (0.024774) with: {'learn_rate': 0.3, 'momentum': 0.4}
0.572917 (0.134575) with: {'learn_rate': 0.3, 'momentum': 0.6}
0.455729 (0.146518) with: {'learn_rate': 0.3, 'momentum': 0.8}
0.544271 (0.146518) with: {'learn_rate': 0.3, 'momentum': 0.9}
```
我们可以看到 SGD 在这个问题上表现得并不是特别好，尽管在学习速率为0.001，动量为 0.4 的情况下获得了 67% 的精确度。

## 权重初始化

神经网络权重初始化被简单的运用：使用较小的随机值。 现在可以从 [Keras](http://keras.io/initializations/) 提供的一组列表来选择不同的初始化技术。 

在这个案例中，我们会评估所有可用的技术来调节网络权重初始化选择。在每一层我们会选择相同的权重初始化方法。理想的情况下，最好在每一层，依据激活函数的不同使用不同的初始化方案。在下面的案例中，在隐藏层我们使用 relu 激活函数，在输出层使用 sigmoid 函数，因为预测是二分类。代码如下：

In [ ]:
# 构建模型的函数
def create_model(init_mode='uniform'):
    # 创建模型
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    
    # 编译模型
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型，使用到了上一步找出的 epochs、batch size 最优参数
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=80, verbose=0)
# 定义网格搜索参数
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 
             'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

运行的结果如下：
```
Best: 0.709635 using {'init_mode': 'uniform'}
0.709635 (0.018136) with: {'init_mode': 'uniform'}
0.678385 (0.003683) with: {'init_mode': 'lecun_uniform'}
0.703125 (0.030425) with: {'init_mode': 'normal'}
0.651042 (0.024774) with: {'init_mode': 'zero'}
0.656250 (0.003189) with: {'init_mode': 'glorot_normal'}
0.680990 (0.015733) with: {'init_mode': 'glorot_uniform'}
0.558594 (0.158416) with: {'init_mode': 'he_normal'}
0.529948 (0.134575) with: {'init_mode': 'he_uniform'}
```
我们看到最好的结果是通过'uniform' 初始哈权重获得的，获得了 70% 的表现。

## 激活函数

激活函数控制着神经元的非线性在激活时。通常，relu 激活函数是最经常使用的，但对于不同的问题，sigmoid 和 tanh 这些函数可能更合适。

在这案例中，我们会评估一组 [keras](http://keras.io/activations/) 提供的一组激活函数。我们只在隐藏层使用这些激活函数，因为在输出时我们需要 sigmoid 的函数为了二分类问题。通常，为不同的转换函数准备不同的数据是更好的，在这个案例中我们不会这么做。示例代码如下:

In [ ]:
# 构建模型的函数
def create_model(activation='relu'):
    # 创建模型
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # 编译模型
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型，使用到了上一步找出的 epochs、batch size 最优参数
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=80, verbose=0)
# 定义网格搜索参数
activation = ['softmax', 'softplus', 'softsign', 'relu', 
              'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid=dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

得到的结果如下：
```
Best: 0.713542 using {'activation': 'softplus'}
0.657552 (0.028940) with: {'activation': 'softmax'}
0.713542 (0.012075) with: {'activation': 'softplus'}
0.683594 (0.016877) with: {'activation': 'softsign'}
0.691406 (0.005524) with: {'activation': 'relu'}
0.694010 (0.021236) with: {'activation': 'tanh'}
0.680990 (0.025582) with: {'activation': 'sigmoid'}
0.669271 (0.028587) with: {'activation': 'hard_sigmoid'}
0.701823 (0.016053) with: {'activation': 'linear'}
```
很惊奇，'softplus' 激活函数获得了最好的结果（在实践中 relu 使用的更多），达到了 71%。

## Dropout

在这个案例中，我们考虑调节 Dropout 为了正则化，他是为了限制过拟合以及提供模型的泛化能力。

为了获得最好的效果，Dropout 最好联合权重约束（weight constraint ），如 max norm constraint。更过在 Keras 的深度模型中使用 Dropout 的信息可以参考此文：[Dropout Regularization in Deep Learning Models With Keras](http://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/) 。这涉及到拟合 Dropout 的百分比和权重约束。我们会尝试 Dropout 的百分比在 0.0 到 0.9 之间（1.0没有意义），同时 max norm weight 约束在 0 到 5 之间。示例代码如下：

In [ ]:
from keras.layers import Dropout
from keras.constraints import maxnorm
# 构建模型的函数
def create_model(dropout_rate=0.0, weight_constraint=0):
    # 创建模型
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # 编译模型
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型，使用到了上一步找出的 epochs、batch size 最优参数
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=80, verbose=0)
# 定义网格搜索参数
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(weight_constraint=weight_constraint, dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

代码运行的结果如下：
```
Best: 0.717448 using {'dropout_rate': 0.2, 'weight_constraint': 1}
0.704427 (0.016367) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.692708 (0.008027) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.707031 (0.006379) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.688802 (0.001841) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.701823 (0.012890) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.701823 (0.007366) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.701823 (0.014382) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.701823 (0.016053) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.701823 (0.001841) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.707031 (0.011049) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.717448 (0.016053) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.695313 (0.011049) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.691406 (0.008438) with: {'dropout_rate': 0.2, 'weight_constraint': 3}
0.701823 (0.011201) with: {'dropout_rate': 0.2, 'weight_constraint': 4}
0.697917 (0.008027) with: {'dropout_rate': 0.2, 'weight_constraint': 5}
0.707031 (0.020915) with: {'dropout_rate': 0.3, 'weight_constraint': 1}
0.683594 (0.022326) with: {'dropout_rate': 0.3, 'weight_constraint': 2}
0.707031 (0.028705) with: {'dropout_rate': 0.3, 'weight_constraint': 3}
0.704427 (0.004872) with: {'dropout_rate': 0.3, 'weight_constraint': 4}
0.686198 (0.009744) with: {'dropout_rate': 0.3, 'weight_constraint': 5}
0.710938 (0.035084) with: {'dropout_rate': 0.4, 'weight_constraint': 1}
0.708333 (0.012890) with: {'dropout_rate': 0.4, 'weight_constraint': 2}
0.701823 (0.009744) with: {'dropout_rate': 0.4, 'weight_constraint': 3}
0.691406 (0.006379) with: {'dropout_rate': 0.4, 'weight_constraint': 4}
0.701823 (0.024150) with: {'dropout_rate': 0.4, 'weight_constraint': 5}
0.687500 (0.008438) with: {'dropout_rate': 0.5, 'weight_constraint': 1}
0.691406 (0.020915) with: {'dropout_rate': 0.5, 'weight_constraint': 2}
0.690104 (0.016367) with: {'dropout_rate': 0.5, 'weight_constraint': 3}
0.686198 (0.022628) with: {'dropout_rate': 0.5, 'weight_constraint': 4}
0.688802 (0.025976) with: {'dropout_rate': 0.5, 'weight_constraint': 5}
0.696615 (0.006639) with: {'dropout_rate': 0.6, 'weight_constraint': 1}
0.703125 (0.016573) with: {'dropout_rate': 0.6, 'weight_constraint': 2}
0.696615 (0.019488) with: {'dropout_rate': 0.6, 'weight_constraint': 3}
0.695312 (0.022999) with: {'dropout_rate': 0.6, 'weight_constraint': 4}
0.678385 (0.019488) with: {'dropout_rate': 0.6, 'weight_constraint': 5}
0.667969 (0.036782) with: {'dropout_rate': 0.7, 'weight_constraint': 1}
0.677083 (0.041504) with: {'dropout_rate': 0.7, 'weight_constraint': 2}
0.684896 (0.028764) with: {'dropout_rate': 0.7, 'weight_constraint': 3}
0.674479 (0.030978) with: {'dropout_rate': 0.7, 'weight_constraint': 4}
0.669271 (0.012890) with: {'dropout_rate': 0.7, 'weight_constraint': 5}
0.653646 (0.021236) with: {'dropout_rate': 0.8, 'weight_constraint': 1}
0.651042 (0.024774) with: {'dropout_rate': 0.8, 'weight_constraint': 2}
0.661458 (0.020505) with: {'dropout_rate': 0.8, 'weight_constraint': 3}
0.661458 (0.023073) with: {'dropout_rate': 0.8, 'weight_constraint': 4}
0.651042 (0.024774) with: {'dropout_rate': 0.8, 'weight_constraint': 5}
0.651042 (0.024774) with: {'dropout_rate': 0.9, 'weight_constraint': 1}
0.651042 (0.024774) with: {'dropout_rate': 0.9, 'weight_constraint': 2}
0.651042 (0.024774) with: {'dropout_rate': 0.9, 'weight_constraint': 3}
0.652344 (0.025315) with: {'dropout_rate': 0.9, 'weight_constraint': 4}
0.651042 (0.024774) with: {'dropout_rate': 0.9, 'weight_constraint': 5}
```
我们可以看到 dropout 设置为 0.2 同时 maxnorm 权重约束设置为 1 或者最好的结果，精确度在 72%

## 隐藏层的神经元数

层中的神经元数也是一个非常重要的需要调节的参数。通常在层中的神经元数量控制了网络的表示能力，至少在那一点在拓扑中。

虽然，一般一个大的单层网络可以近似任何其他神经网络，至少在[理论上](https://en.wikipedia.org/wiki/Universal_approximation_theorem)。在这个案例中，我们考虑调节在当个隐藏层中的神经元的个数。我们会尝试的值从 1 到 30 间隔为 5 。 一个大的网络需要更多的训练时间，至少 batch size 和 epochs 应该被优化通过神经元的个数。示例代码如下：

In [ ]:
from keras.layers import Dropout
from keras.constraints import maxnorm
# 构建模型的函数
def create_model(neurons=1):
    # 创建模型
    model = Sequential()
    model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # 编译模型
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 为了复现，设置随机种子
seed = 7
np.random.seed(seed)

# 加载数据
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 切分数据为输入 X 和输出 Y
X = dataset[:,0:8]
Y = dataset[:,8]

# 创建模型，使用到了上一步找出的 epochs、batch size 最优参数
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=80, verbose=0)
# 定义网格搜索参数
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 总结结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

运行结果如下：
```
Best: 0.705729 using {'neurons': 25}
0.665365 (0.035849) with: {'neurons': 1}
0.695312 (0.022326) with: {'neurons': 5}
0.699219 (0.011500) with: {'neurons': 10}
0.701823 (0.016053) with: {'neurons': 15}
0.700521 (0.001841) with: {'neurons': 20}
0.705729 (0.013279) with: {'neurons': 25}
0.696615 (0.006639) with: {'neurons': 30}
```
可以看到在隐藏层含有 5 个神经元是精确度达到了 71%

# 超参优化技巧

这一张列出了一些遍历的技巧当你调节你的神经网络超参时。
- **k 折交叉验证**：这篇文章的案例中，你可以看到结果中展示了一些方差。默认的 3 折交叉验证被使用，但也许 k=5 或者 k=10会是的结果更加稳定。小心的选择你的交叉验证配置以确保你的结果是稳定的。
- **检查整个网格**：不要只盯着最优的结果，检查整个网格的结果，寻找支持配置决策的趋势
- **并行**：如果可能使用全部的内核，神经网络训练很慢，通常我们又会想尝试很多不同的参数。
- **使用数据的一个采样**：因为神经网络训练很慢，尝试在训练集的小采样上训练他们，仅仅是获得一般的参数方向而不是优化的配置
- **从粗糙的网格开始**：从粗略的网格开始，一旦缩小了范围，就可以缩放到细粒度上。
- **不要转换结果**：结果通常针对特定的问题。试着避免偏好配置在每一个新的问题上。这不像你在一个问题上发现的最优结果可以转换到你的下一个项目上。相反是寻找更广阔的趋势，像层数或者参数之间的关系
- **问题的重现**：虽然我们设置了种子为 numpy 中的随机数生成器，但结果并不是可以百分百的复现。当网格搜索包含 Keras 模型时，复现较比文中呈现的更容易。

# 总结

在此文中，你知道了在 python 中使用 keras 和 scikit-learning 如何调节你的深度学习网络的超参。特别的，你学到了：
- 如何包装 Keras 模型并在 scikit-learn 中使用，同时学习如何使用网格搜索
- 如何为 Keras 模型网格搜索一套不同标准的神经网络参数
- 如何设计你自己的超参调节实验

你是否有调节大型神经网络超参的经验，请在下面分享你的故事。

你是否对神经网络的超参优化有什么问题，或则关于这篇文章？在评论中留下你的问题，我会尽我最大的努力回答。